# Homework 7: Spreadsheet

Due: 11/29

In a module `spreadsheet.py` create a new `tkinter`-based class called `Spreadsheet` that is a `Frame` that contains a labelled array of cells that looks something like this (including a focus `Label` and a focus `Entry` -- see below):

<img src="spreadsheet.png" alt="Spreadsheet illustration">

In this figure, the focus `Label` is on the left of the bottom row (containing "`a0:`") and the focus `Entry` (empty in this case) is to its right.

The prototype for a `Spreadsheet` instantiation is:
``
Spreadsheet(parent, nr=4, nc=4)
``
where `nr` is the number of rows and `nc` is the number of columns.
The columns are numbered left to right from `0` to `nc-1`.
The `nr` rows are lettered in lower case starting with `a` in
on the top.
(Don't worry about having more than 26 rows for now!)

Let's demonstrate this. The solution module contains a self-test that creates a 4x4 spreadsheet:

In [8]:
!python3 spreadsheet.py

The interaction model for this spreadsheet differs from that of normal Excel$^{TM}$-like (actually, VisiCalc$^{TM}$-like) spreadsheets, and may actually be easier for a programmer (especially a Python
programmer) to understand.  In "normal" spreadsheets, cells have text values or numeric values.  In this case, all cells contain strings which can be evaluated like any Python expression. Strings evaluate to themselves and can therefore be used as both data and labels.

Cell expressions can use the names of other cells, with the exception that a cell cannot refer to itself either directly or indirectly. We'll demonstrate this.

The suggested implementation is to give each cell three attributes:

- an `expression`: a string the user types into the focus `Entry` (below)

- a `code`: a code (that's a class) object created by the `compile()` builtin), and

- a `value`: the result of evaluating the `code` (using `eval()` and, converted to a string (using `str()`), is what the users see when they look at a cell.

At any given time, one cell is the "focus". It appears with a yellow background. Associated with the focus are two additional widgets: the "focus `Label`" and the "focus `Entry`". The focus `Entry` is where the user edits the expression string for the focus and the focus Label is the name (`"a0"`, `"c3"`, etc.) of the focus cell. They are attributes (`focusLabel` and `focusEntry`) of the `Spreadsheet` and must be placed (e.g. via `grid()`) separately from the `Spreadsheet` widget itself.

It is strongly recommended that the cells be placed within the `Spreadsheet` using the `grid` geometry manager.

The user can click on any cell in the cell array to make it the focus. The focus Entry shows that cell's expression string and the focus `Label` shows its label. Pressing the `Enter` or `Tab` keys causes the focus `Entry` to become the cell's expression string, which is then compiled to a new code. After that, the cell and all cells related to it are updated to reflect the new value.

As states above, the cell array (i.e., the `Spreadsheet`), the focus `Label`, and the focus `Entry` all need to be placed with a geometry manager. Here is a test program (which is also the `spreadsheet` module's self-test) that your implementation should support:

In [29]:
# %load spreadsheet_t.py
from tkinter import Tk
from spreadsheet import Spreadsheet

root = Tk()
root.title("Spreadsheet Self-Test")
nRows = 4
nCols = 4
spreadsheet = Spreadsheet(root, nRows, nCols)
spreadsheet.grid(row=0, column=0, columnspan=nCols)
spreadsheet.focusLabel.grid(row=1, column=0)
spreadsheet.focusEntry.grid(row=1, column=1)
root.mainloop()


This is downloadable from the web page, along with a working `spreadsheet.cpython-36.pyc` to compare against your work. The grader may have one or more tests that differ from this, but your code should at least work with this.

Note that `spreadsheet.cpython-36.pyc` is a bytecode-compiled version of the instructor's solution that can be imported via "`import spreadsheet`" or invoked from the command line like any other Python module:

~~~~
$ python3 spreadsheet.cpython-36.pyc
~~~~

but, as the name suggests, it only works for Python 3.6 (the official version of this class). On the other hand it should work on any platform running Python 3.6, since Python bytecodes are platform-independent.

## Compiling and Evaluating Cells

Initially, all cells start with a expression `""` (an empty string) and a code of `None`, so all cells initially appear blank.

To show how evaluation works, let's take an example. Let's suppose we want to have an expression in cell `a2` that adds the contents of cells `a0` and `a1`, which we set to 1 and 2, respectively. We would enter this expression in `a2`:

In [10]:
expr = "a0 + a1"

When this expression is entered, we use `compile()` to create a `code` class object that represents it:

In [11]:
code = compile(expr, "a2", 'eval')

In order to evaluate `code`, we must provide a dictionary that maps the symbols `expr` uses to values. We call this a "symbol table":

In [12]:
symtab = { 'a0': 1, "a1": 2 }

and then pass this to `eval()` to get a value:

In [13]:
value = eval(code, {}, symtab)
value

3

Passing `symtab` as the third argument to `eval()` makes it refer to its symbols as global, rather than local. This is what we want. (Local would make sense if we were inside a function or method, which we are not.)

When we display this in a `Cell` (which is a `Label`) we would use `str()` to convert it to a string (if necessary):

In [14]:
str(value)

'3'

## Adding More Symbols to the Symbol Table

We can add other symbols to symtab and they can map to objects such as functions and constants:

In [15]:
def f(x):
    return x**2 + 3
symtab['f'] = f
from math import sin, pi
symtab['sin'] = sin
symtab['pi'] = pi
expr = 'f(a0) + sin(pi/4)'
code = compile(expr, 'a3', 'eval')
eval(code, {}, symtab)

4.707106781186548

Your `Spreadsheet` should allow users to use all of the functions and constants (e.g.`pi`) in the `math` module. You can find them in the `module.__dict__` dictionary, but ignore those symbols that begin with `'_'`.

## Dealing With Dependencies

When the user changes a cell's expression, it's not enough to update the cell itself. We also need to update all cells that depend on that cell. This is a problem in graph theory. Solving it is beyond the range of this course, so I've provided a `dependencies` module to do that for you. (You're free to write your own, of course!)

Here's how it works:

Set up a dictionary (*not* the symbol table) that contains the dependencies of each cell by name. If cell `a2` is has the expression "`a0 + a1`" and cell `a3` has the expression "`f(a0) + sin(pi/4)`", the dependencies would be:

In [19]:
deps = { 'a2': ('a0', 'a1'), 'a3': ('a0',)}

There's only one function in `dependencies` you should need: `dependersOn()`:

In [20]:
from dependencies import dependersOn

In [18]:
?dependersOn

Pass it any cell's name and it will return a list of names of those cells that depend on it and therefore need to be updated:

In [21]:
dependersOn('a0', deps)

['a2', 'a3']

In [22]:
dependersOn('a1', deps)

['a2']

In [23]:
dependersOn('a2', deps)

[]

So all you need is to traverse the list (in order) and update the cells' values in both the symbol table and on the screens.

### Finding Dependencies

It will be up to you to build the dependencies dictionary (`deps` in the above examples). You *could* parse the expression yourself, but there's a much easier way to do this. When you compile a code object (with `compile()`, of course), it has a number of attributes:

In [24]:
dir(code)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'co_argcount',
 'co_cellvars',
 'co_code',
 'co_consts',
 'co_filename',
 'co_firstlineno',
 'co_flags',
 'co_freevars',
 'co_kwonlyargcount',
 'co_lnotab',
 'co_name',
 'co_names',
 'co_nlocals',
 'co_stacksize',
 'co_varnames']

The one we want is `co_names`:

In [25]:
code.co_names

('f', 'a0', 'sin', 'pi')

This `code` was produced for `a3` (see above) and dependencies only need the names of cells, not the other stuff we put in the symbol table, so we remove everything that isn't a cell name, the appropriate dependency for `a3` would be `(a0,)`.

In [26]:
deps['a3'] = ('a0', )
deps

{'a2': ('a0', 'a1'), 'a3': ('a0',)}

### Cyclic Dependencies

There's one caveat. If you pass a dependency dictionary that includes a cycle, it will raise a `CyclicDependency` exception:

In [28]:
cyclicDeps = { 'a': ('b',), 'b': ('a',) }
dependersOn('a', cyclicDeps)

CyclicDependency: dependency cycle detected

Your code can trap this (or any other exception like `DivideByZero`) when it evaluates a new cell expression and undo the update by saving the initial cell state beforehand and restoring it in the `except` suite.